In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 🎯 AGI Voice Agent - Multimodal Pipeline (Notebook 10)

### Objective:
- Process **Voice → Text** using Whisper
- Process **Image → Caption/Text** using BLIP
- Process **Text → Reasoning** via LLM
- Return **Voice output** via ElevenLabs






✅ 1. Install Dependencies

pip install transformers sentence-transformers whisper elevenlabs Pillow




✅ 2. Load Models

import whisper
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from PIL import Image
from elevenlabs import generate, set_api_key

whisper_model = whisper.load_model("base")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
llm = pipeline('text-generation', model='gpt2')

set_api_key("YOUR_ELEVENLABS_API_KEY")




✅ 3. Process Voice Input → Text

def transcribe_voice(audio_path):
    result = whisper_model.transcribe(audio_path)
    return result['text']




✅ 4. Process Image Input → Caption/Text

def image_to_text(image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = blip_processor(image, return_tensors="pt")
    output = blip_model.generate(**inputs)
    caption = blip_processor.decode(output[0], skip_special_tokens=True)
    return captio


✅ 5. Reasoning: Text Input → Reasoned Output

def reason_text(input_text):
    response = llm(input_text, max_length=100)
    return response[0]['generated_text']




✅ 6. Generate Voice Output

def text_to_voice(text, filename='outputs/voice_output.mp3'):
    audio = generate(text=text, voice="Rachel")
    with open(filename, 'wb') as f:
        f.write(audio)
    return filename




✅ 7. Full Multimodal Handler

def multimodal_pipeline(audio_input=None, image_input=None, text_input=None):
    collected_text = ""

    if audio_input:
        collected_text += transcribe_voice(audio_input) + " "

    if image_input:
        collected_text += image_to_text(image_input) + " "

    if text_input:
        collected_text += text_input

    reasoning = reason_text(collected_text)
    voice_file = text_to_voice(reasoning)
    
    return {
        "input_summary": collected_text.strip(),
        "reasoning_output": reasoning,
        "voice_output": voice_file
    }





✅ 8. Example Usage

result = multimodal_pipeline(
    audio_input="samples/sample_audio.wav",
    image_input="samples/sample_image.jpg",
    text_input="Explain what you see and hear."
)

print("Input Summary:", result['input_summary'])
print("Reasoning:", result['reasoning_output'])
print("Voice Output File:", result['voice_output'])

SyntaxError: invalid character '→' (U+2192) (3097947343.py, line 4)